# Домашняя работа №1

В этой домашней работе вам предлагается ознакомиться с базовой функциональностью Python и устройством Jupyter Notebook, а так же познакомиться с простыми функциями из пакетов NumPy и matplotlib.

Следуйте инструкциям нотебука, решайте задачи, а ответы заполняйте в следующую форму: https://forms.gle/gxG8D5BGeH1nxcSU8

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tasks import find_fold_number, rle, test_rle_str

# увеличим изначальный размер графиков
plt.figure(figsize=(15, 10))

# отображение графиков внутри нотебука
%matplotlib inline
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
# автоматическая перезагрузка функций из локальных модулей
%load_ext autoreload
%autoreload 2

# NumPy & matplotlib

Для следующих заданий необходимо реализовать код внутри ячейки. Постарайтесь воспользоваться функциональностью пакетов, избегая ненужных циклов и т.п.

Про `NumPy` можно почитать тут:
[NumPy quickstart](https://docs.scipy.org/doc/numpy/user/quickstart.html)

Про `matplotlib` тут:
[PyPlot tutorial](https://matplotlib.org/tutorials/introductory/pyplot.html)

В данной части задания будут базироваться на датасете $\textit{Ирисы Фишера}$, первым делом его необходимо скачать

In [17]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -P assignment_1/data

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2020-03-01 18:22:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Распознаётся archive.ics.uci.edu... 128.195.10.252
Устанавливается соединение с archive.ics.uci.edu|128.195.10.252|:443... соединение установлено.
Невозможно установить соединение SSL.


Ирисы Фишера состоят из данных о 150 экземплярах ириса — Ирис щетинистый (Iris setosa), Ирис виргинский (Iris virginica) и Ирис разноцветный (Iris versicolor). Для каждого экземпляра измерялись четыре характеристики (в сантиметрах):

1. Длина наружной доли околоцветника (англ. sepal length);
2. Ширина наружной доли околоцветника (англ. sepal width);
3. Длина внутренней доли околоцветника (англ. petal length);
4. Ширина внутренней доли околоцветника (англ. petal width).

In [5]:
iris_full = np.genfromtxt('iris.data', delimiter=',', dtype='object')
names = ('sepallength', 'sepalwidth', 'petallength', 'petalwidth', 'species')
iris_vals = iris_full[:, :-1].astype(np.float)
iris_name = iris_full[:, -1].astype(np.str)
n_iris = iris_vals.shape[0]

In [27]:
n_rows = 10
template = '{:^15}' * len(names)
print(template.format(*names))
for vals, name in zip(iris_vals[:n_rows], iris_name[:n_rows]):
    print(template.format(*vals, name))

  sepallength    sepalwidth     petallength    petalwidth       species    
      5.1            3.5            1.4            0.2        Iris-setosa  
      4.9            3.0            1.4            0.2        Iris-setosa  
      4.7            3.2            1.3            0.2        Iris-setosa  
      4.6            3.1            1.5            0.2        Iris-setosa  
      5.0            3.6            1.4            0.2        Iris-setosa  
      5.4            3.9            1.7            0.4        Iris-setosa  
      4.6            3.4            1.4            0.3        Iris-setosa  
      5.0            3.4            1.5            0.2        Iris-setosa  
      4.4            2.9            1.4            0.2        Iris-setosa  
      4.9            3.1            1.5            0.1        Iris-setosa  


In [54]:
len(iris_vals)

150

## 1. Какое максимальное значение для каждого признака
Ответом будет последовательность из 4 чисел  
Например: `5.1 3.5 1.4 0.2`

In [42]:
from functools import reduce
maxs = np.zeros((4,1))
for i in range(len(names)-1):
    maxs[i] = max(iris_vals[:, i])
print(maxs)
    

[[7.9]
 [4.4]
 [6.9]
 [2.5]]


## 2. Сколько каждого типа ириса представленно в данных
Ответом будет последовательность из 3 чисел в порядке: `Iris-setosa, Iris-versicolor, Iris-virginica`  
Например: `10 10 10`

In [53]:
species_num = {"Iris-setosa" : 0, "Iris-versicolor" : 0, "Iris-virginica" : 0 }
for name in iris_name:
    species_num[name] += 1
print(species_num)

{'Iris-setosa': 50, 'Iris-versicolor': 50, 'Iris-virginica': 50}


## 3. Среднее значение признака `petalwidth` для каждого типа ириса
Ответом будет последовательность в возврастающем порядке с округлением до 2 знаков после запятой.  
Например: `1.23 4.56 7.89` 

In [69]:
petalwidth_avg = {"Iris-setosa" : 0, "Iris-versicolor" : 0, "Iris-virginica" : 0 }
for iris_val, name in zip(iris_vals[:,3], iris_name):
    petalwidth_avg[name] += iris_val / species_num[name]

for item in petalwidth_avg.items():
    print(round(item[1], 2))

0.24
1.33
2.03


## 4. Попарное скалярное произведение признаков
Ответом будет среднее значение попарных скалярных произведений векторов признаков с округлением до 2 знаков после запятой.  
Скалярное произведение вектора с самим собой учитывать не надо.  
Например: `12.34`

In [104]:
from itertools import *

avg_inner_prod = 0
for i in range(len(names)-1):
    for j in range(len(names)-1):
        if (i != j):
            avg_inner_prod += np.dot(iris_vals[:,i], iris_vals[:,j]) / np.math.factorial(len(names)-2)
print(round(avg_inner_prod, 2) / 2)

1726.215


## 5. У какого типа ириса самое маленькое значение признака `sepalwidth`

In [107]:
min_sepalwidth = iris_vals[0,1]
for iris_val, name in zip(iris_vals[:,1], iris_name):
    min_sepalwidth = min(min_sepalwidth, iris_val)
    if min_sepalwidth == iris_val:
        name_min = name
print(name_min)

Iris-versicolor


## 6. Постройте график распределения значений для `petallength` взависимости от типа ириса
В качестве ответа укажите тип ириса с наименьшей дисперсией (самый "узкий")

## 7. Отобразите зависимость между `petallength` и `petalwidth` взависимости от типа ириса
по оси X - petallength   
по оси Y - petalwidth  
В ответе укажите класс, который отделился от остальных

## 8. Постройте boxplot признака `sepallength` для каждого типа Ириса
В ответе укажите количество выбросов в данных  
Каждый выброс отображается точкой, подробнее про boxplot можно почитать [здесь](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51)

# Python
Для следующих заданий необходимо реализовать соответствующую функцию в файле `tasks.py`.

После реализации, выполните соответствующую ячейку, не изменяя её содержимое.

## 9. За сколько шагов можно получить однозначное число перемножая цифры предыдущего числа.
Например, для $88$ ответ $3$:
$$
88 \rightarrow 8 \times 8 = 64 \rightarrow 6 \times 4 = 24 \rightarrow 2 \times 4 = 8
$$
Для этого задания, реализуйте функцию `find_fold_number`

In [ ]:
assert find_fold_number(88) == 3, "неправильный ответ для числа из примера"

В качестве ответа укажите результат выполнения следующей ячейки (без кавычек)

In [ ]:
''.join(map(str, (find_fold_number(i) for i in range(500))))

## 10. Кодирование длин серии
Кодирование длин серий (RLE) — алгоритм сжатия данных, заменяющий повторяющиеся символы на один символ и число его повторов. Серией называется последовательность, состоящая из нескольких одинаковых символов (более одного). При кодировании строка одинаковых символов, составляющих серию, заменяется строкой, содержащей сам повторяющийся символ и количество его повторов.

Например, для $\textit{AAAAAAAAAAAAAAABAAAA}$ будет сжата в $\textit{A15BA4}$

Для этого задания реализуйте функцию `rle`

In [ ]:
assert rle('AAAAAAAAAAAAAAABAAAA') == 'A15BA4', "неправильный ответ для строки из примера"

В качестве ответа укажите результат выполнения следующей ячейки (без кавычек)

In [ ]:
rle(test_rle_str)